In [1]:
import sys
sys.path.append('/home/UR/kontaous/Bureau/Stage/')
from Analyses.traitement import *
import pandas as pd
import numpy as np
from Analyses.traitement import *


In [2]:
tracking = pd.read_csv("../../../Tracking_data/MontPellier(2)_(2)HAC.csv")
events = pd.read_csv("../../../Tracking_data/events_hac.csv")
tracking, events = prepare_data(tracking, events)
attributs = ['team.name', 'period', 'player.name', 'position.name', 'location.x', 'location.y', 'pass.recipient.name', 'pass.length', 'pass.outcome.name', 'duel.type.name', 'duel.outcome.name', 'interception.outcome.name', 'ball_receipt.outcome.name', 'shot.end_location.x', 'shot.end_location.y', 'shot.outcome.name', 'dribble.outcome.name',]
df = events[attributs].copy()
df.reset_index(drop=True, inplace=True)
df.index = df.index + 1
position_mapping = {
    'Defender': ['right back', 'left back', 'center back', 'right center back', 'left center back', 'right wing back', 'left wing back'],
    'Midfielder': ['center attacking midfield', 'center defensive midfield', 'left center midfield', 'right center midfield', 'left defensive midfield', 'right defensive midfield'],
    'Forward': ['center forward', 'left center forward', 'right center forward', 'right wing', 'left wing']
}
inverse_mapping = {pos: general_pos for general_pos, positions in position_mapping.items() for pos in positions}
df['position.name'] = df['position.name'].fillna('').str.lower()
df['general_position'] = df['position.name'].map(inverse_mapping).fillna(df['position.name'])
df['position.name'] = df['general_position']
df = df.drop(columns=['general_position'])
df

/tmp/ipykernel_8235/2238170368.py:2: DtypeWarning: Columns (30,42,46,48,49,50,51,65,72,87,89,90,91,92,105,109,111,112,116,117,118,119,120,121,124,125,127,129,135,136,137,138,139,140,141,142,144,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  events = pd.read_csv("../../../Tracking_data/events_hac.csv")


,team.name,period,player.name,position.name,location.x,location.y,pass.recipient.name,pass.length,pass.outcome.name,duel.type.name,duel.outcome.name,interception.outcome.name,ball_receipt.outcome.name,shot.end_location.x,shot.end_location.y,shot.outcome.name,dribble.outcome.name
1,Montpellier,1,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Le Havre,1,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Lille,1,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Le Havre,1,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Lille,1,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37142,Montpellier,1,Joris Chotard,Midfielder,106.7,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37143,Montpellier,1,Joris Chotard,Midfielder,109.9,8.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Incomplete
37144,Le Havre,1,Issa Soumaré,Forward,10.2,71.2,NaN,NaN,NaN,Tackle,Lost Out,NaN,NaN,NaN,NaN,NaN,NaN
37145,Le Havre,1,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df.dtypes

datetime                     datetime64[ns]
team.name                            object
period                                int64
player.name                          object
position.name                        object
location.x                          float64
location.y                          float64
pass.recipient.name                  object
pass.length                         float64
pass.outcome.name                    object
duel.type.name                       object
duel.outcome.name                    object
interception.outcome.name            object
foul_committed.type.name             object
ball_receipt.outcome.name            object
shot.end_location.x                 float64
shot.end_location.y                 float64
shot.outcome.name                    object
dribble.outcome.name                 object
dtype: object

In [19]:
df['shot.outcome.name'].unique()

array([nan, 'Off T', 'Wayward', 'Saved', 'Blocked', 'Goal', 'Post'],
      dtype=object)

In [12]:
def evaluate_performance(df, team_name, position):

    df['team.name'] = df['team.name'].astype(str)
    df['position.name'] = df['position.name'].astype(str)
    
    df_filtered = df[(df['team.name'] == team_name) & (df['position.name'] == position)]
    
    if position == 'Defender':
        stats = df_filtered.groupby(['team.name', 'player.name']).agg(
            passes_effectuees=('pass.length', 'count'),
            longueur_moyenne_passes=('pass.length', 'mean'),
            passes_progressives=('pass.recipient.name', 'count'),
            dribbles_reussis=('dribble.outcome.name', lambda x: (x == 'Complete').sum()),
            ballons_interceptes_succes=('ball_receipt.outcome.name', lambda x: (x == 'Incomplete').sum()),
            duels=('duel.type.name', 'count'),
            duels_gagnes=('duel.outcome.name', lambda x: (x.isin(['Success Out', 'Success In Play', 'Won']).sum())),
            duels_perdus=('duel.outcome.name', lambda x: (x.isin(['Lost Out', 'Lost In Play']).sum())),
            interceptions_reussies=('interception.outcome.name', lambda x: (x == 'Won').sum())
        ).reset_index()
        
        stats.columns = ['Équipe', 'Joueur', 'Passes effectuées', 'Longueur moyenne des passes','Passes progressives', 'Dribbles réussis', 'Ballons interceptés avec succès', 'Duels', 'Duels gagnés', 'Duels perdus', 'Interceptions réussies']
    
    elif position == 'Midfielder':
        stats = df_filtered.groupby(['team.name', 'player.name']).agg(
            passes_effectuees=('pass.length', 'count'),
            longueur_moyenne_passes=('pass.length', 'mean'),
            passes_progressives=('pass.recipient.name', 'count'),
            dribbles_reussis=('dribble.outcome.name', lambda x: (x == 'Complete').sum()),
            ballons_interceptes_succes=('ball_receipt.outcome.name', lambda x: (x == 'Incomplete').sum()),
            tirs=('shot.end_location.x', 'count'),
            buts=('shot.outcome.name', lambda x: (x == 'Goal').sum()),
            tirs_cadres=('shot.outcome.name', lambda x: (x.isin(['Saved', 'Goal', 'Post']).sum())),
            tirs_non_cadres=('shot.outcome.name', lambda x: (x.isin(['Off T', 'Wayward', 'Blocked']).sum())),
        ).reset_index()
        
        stats.columns = ['Équipe', 'Joueur', 'Passes effectuées', 'Longueur moyenne des passes', 'Passes progressives', 'Dribbles réussis', 'Ballons interceptés avec succès', 'Tirs', 'Buts', 'Tirs cadrés', 'Tirs non cadrés']
    
    elif position == 'Forward':
        stats = df_filtered.groupby(['team.name', 'player.name']).agg(
            passes_effectuees=('pass.length', 'count'),
            longueur_moyenne_passes=('pass.length', 'mean'),
            tirs=('shot.end_location.x', 'count'),
            buts=('shot.outcome.name', lambda x: (x == 'Goal').sum()),
            tirs_cadres=('shot.outcome.name', lambda x: (x.isin(['Saved', 'Goal', 'Post']).sum())),
            tirs_non_cadres=('shot.outcome.name', lambda x: (x.isin(['Off T', 'Wayward', 'Blocked']).sum())),
            dribbles_reussis=('dribble.outcome.name', lambda x: (x == 'Complete').sum())
        ).reset_index()
        
        stats.columns = ['Équipe', 'Joueur', 'Passes effectuées', 'Longueur moyenne des passes', 'Tirs', 'Buts', 'Tirs cadrés', 'Tirs non cadrés', 'Dribbles réussis']
    
    else:
        stats = pd.DataFrame()
    
    return stats
df_milieu= evaluate_performance(df, "Le Havre", "Midfielder")
df_milieu
df_attaquants= evaluate_performance(df, "Le Havre", "Defender")
df_attaquants

,Équipe,Joueur,Passes effectuées,Longueur moyenne des passes,Passes progressives,Dribbles réussis,Ballons interceptés avec succès,Duels,Duels gagnés,Duels perdus,Interceptions réussies
0,Le Havre,Arouna Sangante,435,24.306768,426,0,6,24,7,4,5
1,Le Havre,Christopher Operi,402,20.408893,373,2,25,19,6,7,0
2,Le Havre,Gautier Lloris,519,20.055478,512,1,16,37,10,10,4
3,Le Havre,Josue Casimir,171,17.965657,169,5,26,11,5,2,3
4,Le Havre,Loïc Négo,116,19.391554,114,5,14,9,7,2,3
5,Le Havre,Oualid El Hajjam,51,21.893221,47,0,0,3,3,0,2
6,Le Havre,Yassine Kechta,0,NaN,0,0,0,0,0,0,0
7,Le Havre,Yoann Salmier,374,21.472649,366,2,12,25,7,7,4
8,Le Havre,Étienne Youte Kinkoue,56,23.101388,56,0,0,0,0,0,0


In [23]:
import dash
from dash import dcc, html, dash_table
import pandas as pd
import plotly.graph_objs as go
from sklearn.preprocessing import MinMaxScaler


def display_player_performance(df, team_name, position, players):
    df_stats = evaluate_performance(df, team_name, position)
    df_selected = df_stats[df_stats['Joueur'].isin(players)]
    
    scaler = MinMaxScaler()
    normalized_stats = pd.DataFrame(scaler.fit_transform(df_selected.iloc[:, 2:]), columns=df_selected.columns[2:])
    normalized_stats.insert(0, 'Joueur', df_selected['Joueur'].values)
    
    data_radar = {'Metrics': normalized_stats.columns[1:].tolist()}
    
    for player in players:
        data_radar[player] = normalized_stats[normalized_stats['Joueur'] == player].iloc[0, 1:].tolist()
    
    df_radar = pd.DataFrame(data_radar)
    
    def create_radar_chart(df):
        
        fig = go.Figure()

        for column in df.columns[1:]:
            fig.add_trace(go.Scatterpolar(
                r=df[column],
                theta=df['Metrics'],
                fill='toself',
                name=column
            ))

        fig.update_layout(
            polar=dict(
                radialaxis=dict(
                    visible=True,
                    range=[0, max(df.iloc[:, 1:].max().max(), 1)]
                )
            ),
            showlegend=True,
            title="Comparaison des Performances des Joueurs"
        )

        return fig
    
    app = dash.Dash(__name__)

    app.layout = html.Div([
        html.H1("Visualisation des Performances des Joueurs"),
        dash_table.DataTable(
            id='table',
            columns=[{"name": i, "id": i} for i in df_selected.columns],
            data=df_selected.to_dict('records'),
            style_cell={'textAlign': 'center'},
            style_header={
                'backgroundColor': 'rgb(230, 230, 230)',
                'fontWeight': 'bold'
            },
            style_data_conditional=[
                {
                    'if': {'row_index': 'odd'},
                    'backgroundColor': 'rgb(248, 248, 248)'
                }
            ],
            style_as_list_view=True
        ),
        dcc.Graph(
            id='radar-chart',
            figure=create_radar_chart(df_radar)
        )
    ])

    if __name__ == '__main__':
        app.run_server(debug=True, port=8051)

display_player_performance(df, 'Le Havre', 'Midfielder', ['Abdoulaye Touré', 'Daler Kuzyaev', 'Yassine Kechta'])
